# Registering Users in Syft Domain Server

By default users are not allowed to create a new account on the Syft Domain Server. This notebook is a tutorial for Data Owners to enable guest signups on their deployments.

### Import packages

In [ ]:
SYFT_VERSION = ">=0.8.1b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
%pip install {package_string} -f https://whls.blob.core.windows.net/unstable/index.html -q

In [ ]:
import syft as sy
sy.requires(SYFT_VERSION)

### Launch a Syft Domain Server

In [ ]:
# Launch a fresh domain server named "test-domain-registration" in dev mode on the local machine
node = sy.orchestra.launch(name="test-domain-1", port="auto", dev_mode=True, reset=True)

In [ ]:
# log into the node with default root credentials
root_client = node.login(email="info@openmined.org", password="changethis")

#### By default registration is disabled. Only `root_client` can register

In [ ]:
# Register a new user using root credentials
response_1 = root_client.register(email="joker@gotham.com", password="joker123", name="Joker")
response_1

In [ ]:
# Register a new user as a GUEST
response_2 = node.register(email="batman@gotham.com", password="1rIzHAx6uQaP", name="Batman")
response_2

In [ ]:
# Register a new user as a GUEST
response_3 = node.register(email="robin@gotham.com", password="5v1ei4OM2N4m", name="Robin")
response_3

In [ ]:
assert root_client.settings.get().signup_enabled == False
assert isinstance(response_1, sy.SyftSuccess)
assert isinstance(response_2, sy.SyftError)
assert isinstance(response_3, sy.SyftError)

#### Now, if root user enable registration, then the guest clients can also register

In [ ]:
# Get the current settings of the node 
root_client.settings.get()

In [ ]:
# Enable guest signups
root_client.settings.allow_guest_signup(enable=True)

In [ ]:
# Refresh the root client to fetch the updated settings
root_client = node.login(email="info@openmined.org", password="changethis")

In [ ]:
# Register a new user as a GUEST
response_2 = node.register(email="batman@gotham.com", password="1rIzHAx6uQaP", name="Batman")
response_2

In [ ]:
# Register a new user as a GUEST
response_3 = node.register(email="robin@gotham.com", password="5v1ei4OM2N4m", name="Robin")
response_3

In [ ]:
assert root_client.settings.get().signup_enabled == True
assert isinstance(response_1, sy.SyftSuccess)
assert isinstance(response_2, sy.SyftSuccess)
assert isinstance(response_3, sy.SyftSuccess)

### Toggle signup again

In [ ]:
# Refresh the root client
root_client.settings.allow_guest_signup(enable=False)

In [ ]:
# Refresh the root client to fetch the updated settings
root_client = node.login(email="info@openmined.org", password="changethis")

In [ ]:
# Register a new user as a GUEST
response_2 = node.register(email="bane@gotham.com", password="SKY5cC2zQPRP", name="Bane")
response_2

In [ ]:
# Register a new user as a GUEST
response_3 = node.register(email="riddler@gotham.com", password="7eVGUuNDyH8P", name="Riddler")
response_3

In [ ]:
assert root_client.metadata.signup_enabled == False
assert isinstance(response_1, sy.SyftSuccess)
assert isinstance(response_2, sy.SyftError)
assert isinstance(response_3, sy.SyftError)

In [ ]:
# Cleanup local domain server

if node.node_type.value == "python":
    node.land()